# Embeddings

Привет! Сегодня ты поработаешь с эмбеддингами: сделаешь классификатор эмоции твитов. Для начала, загрузи их:

In [12]:
#!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
#!unzip archive.zip

Заимпортируй библиотеки и сделай работу скриптов вопсроизводимой.

In [13]:
# conda install gensim=3.4.0 smart_open=1.9.0

In [14]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [15]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [16]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные

In [17]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [18]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Текст очень грязные. Надо добавить очистку текста в его предобработку. 

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [19]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Стокенизируем текст, избавим от знаков пунктуации и мелких слов.

In [20]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [21]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


Загрузим предобученную модель эмбеддингов. Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data . 

In [22]:
word2vec = api.load("word2vec-google-news-300")

/home/arsen/miniconda3/envs/dlenv/lib/python3.7/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [23]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Эмбеддинги не нормализированны, поэтому это тоже надо делать (нейросети и не только любят нормальные данные).

In [24]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

# DATASET

In [25]:
from torch.utils.data import Dataset, random_split
import re

class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, remove_links: bool = True):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column
        
        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)
        
        if remove_links:
            data['text'].replace(r"@\w+\ ", '', regex=True, inplace=True)
            data['text'].replace(r"http://([^\s]+) ", '', regex=True, inplace=True)
            data['text'].replace(r"https://([^\s]+) ", '', regex=True, inplace=True)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokens = self.tokenizer.tokenize(text.lower())
        filtered_tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        return filtered_tokens
    
    def get_embeddings_(self, tokens):
        tokens = [token for token in tokens if token in self.word2vec.vocab]
        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in self.word2vec and len(w) > 3] # Получи эмбеддинги слов и усредни их

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [26]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

/home/arsen/miniconda3/envs/dlenv/lib/python3.7/site-packages/pandas/core/series.py:4569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


## Average embedding
---
Попробуем получить векторное представление предложения из эмбеддингов слов. Самый простой вариант: усреднить вектора по всем словам. Полученный вектор можно отправить любому классификатору как вектор признаков.

Посмотрим, насколько хорошо усреднее работает для определение эмоций твитов. Сделаем их визуализацию.

In [27]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Можно получить более аккуратными алгоритмами, но данный алгоритм покажет сложность задачи и поможет оценить требования к классификатору.

In [28]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(X=examples["features"]) # Обучи PCA на эмбеддингах слов

In [29]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [30]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1003', ...)

Скорее всего, ты увидел не сильно различающиеся классы. Возможно, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую(хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [31]:
from torch.utils.data import DataLoader

batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Для обученния и тестирования нейросетевой модели сделаем отдельные функции.

# TRAINING LOOP

In [32]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device).float()
        
        optimizer.zero_grad()
        outputs = model(features) # Получи предсказания модели
        loss = criterion(outputs, targets) # Посчитай лосс
        loss.backward()
        optimizer.step() # Обнови параметры модели
        
        pbar.set_description(f"Epoch {epoch + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device).float()

            outputs = model(features) # Получи предсказания модели
            loss = criterion(outputs, targets) # Посчитай лосс
            
            acc = ((outputs>0.5).float() == targets.reshape(-1,1)).float().sum()/len(targets) # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Можете создавать любую модель, с любым количеством слоев, функций активации и прочее.


# MODEL

In [33]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 3e-3
num_epochs = 3

layers = []
layers.append(nn.Linear(vector_size, 250))
layers.append(nn.Sigmoid())
layers.append(nn.Linear(250, 200))
layers.append(nn.Sigmoid())
layers.append(nn.Linear(200, 150))
layers.append(nn.Sigmoid())
layers.append(nn.Linear(150, 100))
layers.append(nn.Sigmoid())
layers.append(nn.Linear(100, 50))
layers.append(nn.Sigmoid())
layers.append(nn.Linear(50, num_classes-1))
layers.append(nn.Sigmoid())
model = nn.Sequential(*layers) # Твоя модель

model = model.to(device)
criterion = nn.BCELoss() # Твой лосс
optimizer = Adam(model.parameters(), lr=lr) # Твой оптимайзер

Обучим модель и протестируем её.
После каждой эпохи, проверим качество модели по валидационной части датасета. Если метрика стала лучше, будем сохранять модель. Подумай, какая метрика (точность или лосс) будет лучше работать в этой задаче? 

Здесь и далее реализованно с помощью лосс. Если думаешь, что лучше сравнивать модель через качество, то поменяй код выбора модели.

In [34]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Acc"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Acc"]

/home/arsen/miniconda3/envs/dlenv/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1024])) that is different to the input size (torch.Size([1024, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)



{'Test Loss': 0.5204112453460693, 'Test Acc': 0.7368046875}



{'Test Loss': 0.5033200724124909, 'Test Acc': 0.7512265625}



{'Test Loss': 0.4973659225702286, 'Test Acc': 0.7552109375}


In [35]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

/home/arsen/miniconda3/envs/dlenv/lib/python3.7/site-packages/pandas/core/series.py:4569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,



{'Test Loss': 0.5216771208059293, 'Test Acc': 0.7361908446485623}


/home/arsen/miniconda3/envs/dlenv/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


## TF-iDF
---

Вместо обычного усреднения эмбеддингов их можно дополнительно перевзвесить. Для этого воспользуемся алгоритмом `TD-iDF`. Он уже реализован в библиотеке `scikit-learn`, остается только его добавить в наш пайплайн.

In [36]:
from collections import defaultdict
from typing import Dict

from sklearn.feature_extraction.text import TfidfVectorizer


class TwitterDatasetTfIdf(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, weights: Dict[str, float] = None, remove_links: bool = True):
        super().__init__(data, feature_column, target_column, word2vec, remove_links)
        
    
        if weights is None:
            self.weights = self.get_tf_idf_()
        else:
            self.weights = weights

    def get_embeddings_(self, tokens):
        tokens = [token for token in tokens if token in self.word2vec.vocab]
        embeddings = [(self.word2vec.get_vector(token) - self.mean) / self.std  * self.weights.get(token, 1) for token in tokens]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def get_tf_idf_(self):
        # Надо обучить tfidf на очищенном тексте. Но он принимает только список текстов, а не список списка токенов. Надо превратить второе в первое
        texts = self.data[self.feature_column]
        tokens_list = [self.tokenizer.tokenize(text.lower()) for text in texts]
        filtered_tokens_list = [[w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3] for tokens in tokens_list]
        tokenized_texts = [' '.join(filtered_tokens) for filtered_tokens in filtered_tokens_list]
        tf_idf = TfidfVectorizer()
        # Обучи tf-idf
        tf_idf = tf_idf.fit(tokenized_texts)
        return dict(zip(tf_idf.get_feature_names(), tf_idf.idf_))


In [37]:
dev = TwitterDatasetTfIdf(dev_data, "text", "emotion", word2vec)

Посмотрим на сложность получившейся задачи используя визуализацию через `PCA`.

In [38]:
indexes = np.arange(len(dev))
#np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [39]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(X=examples["features"]) # Ты знаешь, что делать

In [40]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1100', ...)

Создать нейросетку, обучим её на этих данных.

In [41]:
train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

# MODEL

In [42]:
# Не забудь поиграться с параметрами ;)

vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 3e-3
num_epochs = 3

layers = []
layers.append(nn.Linear(vector_size, 250))
layers.append(nn.Sigmoid())
layers.append(nn.Linear(250, 200))
layers.append(nn.Sigmoid())
layers.append(nn.Linear(200, 150))
layers.append(nn.Sigmoid())
layers.append(nn.Linear(150, 100))
layers.append(nn.Sigmoid())
layers.append(nn.Linear(100, 50))
layers.append(nn.Sigmoid())
layers.append(nn.Linear(50, num_classes-1))
layers.append(nn.Sigmoid())
model = nn.Sequential(*layers) # Твоя модель

model = model.to(device)
criterion = nn.BCELoss() # Твой лосс
optimizer = Adam(model.parameters(), lr=lr) # Твой оптимайзер

In [43]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    print(testing(model, criterion, valid_loader, device))
    print(log)
    if log["Test Acc"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Acc"]

/home/arsen/miniconda3/envs/dlenv/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1024])) that is different to the input size (torch.Size([1024, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)



{'Test Loss': 0.5239505858421326, 'Test Acc': 0.73609765625}
{'Test Loss': 0.4973659225702286, 'Test Acc': 0.7552109375}


KeyboardInterrupt: 

In [ ]:
test_loader = DataLoader(
    TwitterDatasetTfIdf(test_data, "text", "emotion", word2vec, weights=dev.weights) , 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Есть ли разница в качестве между способами? Получилось ли улучшить качество модели?

----
нет))))
accuracy на уровне 0.76 на трейне, 0.73 на тесте

----

Сделай небольшое исследование:
- Попробуй сделать несколько нейросеток в качестве классификатора
- Попробуй другие предобученные эмбеддинги
- Попробуй очистить текст от ников ("@username"), url-ов и других символов

Для реализации последнего тебе могут помочь регулярные выражения (`import re`). Напише ниже отчет, что ты попробовал и что получилось.

---

1. очистка от ников и ссылок:  
эффекта ноль
2. другие эмбеддинги:
glove-twitter-25: train acc 0.68
glove-twitter-200: не загрузился
3. разные нейросетки:
3 сетки, fc слои [250,200,150,50], [300,300,300,300], [50,50,50,50], дальше четырех слоев переобучение
---